# Задание
* Собрать информацию о всех строящихся объектах на сайте "наш.дом.рф"
* Cохранить ее в pandas dataframe
* Cохранить pandas dataframe в excel
* Cохранить pandas dataframe в pickle
* Cохранить pandas dataframe в БД

# Задание *
* Написать скрипт, который скачивает фотографии всех строящихся объектов с сайта "наш.дом.рф"


In [93]:
import requests
import pandas as pd

## Список объектов

In [ ]:
#начальное смещение
offset_ = 0

#кол-во записей для загрузки за один запрос
limit_ = 1000

res = []
#загружаем частями все объекты
while 1==1: 
    url = f'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/kn/object?offset={offset_}&limit={limit_}&sortField=devId.devShortCleanNm&sortType=asc&objStatus=0'
    temp = requests.get(url).json().get('data').get('list')
    if len(temp) :        
        res.append(temp)
        offset_ += limit_
    else:
        print('loaded : ' + str(offset_) + ' records')
        break


In [66]:
# собираем DataFrame списка json
df = pd.concat([ pd.json_normalize(x) for x in res ] , ignore_index= True )

In [ ]:
# проверим что результирующий DataFrame содержит все загруженные записи
df.shape[0]

In [90]:
#выгрузим в excel
df.to_excel('./house_data.xlsx')

#выгрузим в pickle
df.to_pickle('./house_data.pkl')

In [ ]:
#выгрузим в БД
import sqlite3

#создаем базу
conn = sqlite3.connect('sample.db')
#выгружаем данные в таблицу house_data
df.to_sql('house_data', conn , if_exists='replace')

conn.commit()
conn.close()


# Сохранение фото по списку объектов с сайта нащ.дом.рф

In [357]:
#количество объектов для которых необходимо сохранить все фото
OBJ_CNT = 3
#путь для сохранения фото
SAVE_PATH = './'

In [385]:
#функция которая по заданному id объекта сохраняет сохраняет страницу с информацией о фотографиях
def get_page ( obj_id ):
    obj_url= 'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/%D0%BA%D0%B0%D1%82%D0%B0%D0%BB%D0%BE%D0%B3-%D0%BD%D0%BE%D0%B2%D0%BE%D1%81%D1%82%D1%80%D0%BE%D0%B5%D0%BA/%D0%BE%D0%B1%D1%8A%D0%B5%D0%BA%D1%82/' + str(obj_id)
    obj_js = 'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/_next/static/chunks/commons.46eca9a97646c7a3d9cf.js'
    header = {
    'user_agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.71 Safari/537.36'
    }
    session = requests.Session()

    getpage1 = session.get(obj_url, headers=header)
    postrequest = session.post(obj_js, headers=header)
    getpage2 = session.get(obj_url, headers=header)
    session.close()
    return ( getpage2 )

In [384]:
from bs4 import BeautifulSoup as bs
import json
#функция которая по странице с информацией о фото возвращает список [ (id, путь, имя)]
def photo_base(loaded_page):
    #распарсим html для удобства извлечения необходимого элемента
    page_parsed = bs(loaded_page.text, 'html.parser') 
    #сохраняем в json формат елемент в котором находится список с фото
    fmt_json =  page_parsed.find_all(id="__NEXT_DATA__")[0].contents[0]
    res_dict = json.loads(fmt_json) 

    # сохранение списка фото
    photo_list = res_dict.\
                        get('props').\
                        get('initialState').\
                        get('kn').\
                        get('objectCard').\
                        get('constructionProgress').\
                        get('shortInfo')
    
    # формируем кортеж с путями и именми файлов
    res = []
    for i in photo_list:
        res.append((i.get('objId'),i.get('src'),i.get('filename')))
    
    return res

In [ ]:
#вспомогательная функция для поиска нужной иерархии 
def expand_js(js_dict, s ):
    s += '\t'
    if (type(js_dict) == dict):
        for i in js_dict:
            print(s,end='')
            print(i)
            expand_js(js_dict.get(str(i)),s)
    else:
        print(s + str(js_dict))

#поиск нахождения списка с фото
expand_js(res_dict,'')

In [362]:
import wget

In [371]:
#функция сохранения фотографий по переданному списку
def photo_save ( photo_lst, path = SAVE_PATH ):
    cnt = 1
    photo_arr = []
    for i in photo_lst:
        source = i[1]
        dest = path + str(cnt) + '_' + str(i[2])
        file_name = str(cnt) + '_' + str(i[2])
        wget.download(source, dest) 
        photo_arr.append([i[0],path,file_name])
        cnt += 1
    return photo_arr

In [372]:
#список id объектов по которым сохраним фото
catalog = pd.read_pickle('house_data.pkl')
objs = list(catalog['objId'].head(OBJ_CNT))

In [386]:
result = []
for id in objs:
    curr_page = get_page(id)
    photo_list = photo_base(curr_page)
    result += photo_save(photo_list , './photo/')


In [389]:
#сохраним датафрейм с объектами и информацией о расположении фото
df = pd.DataFrame(result, columns=['obj_id','path','file'])
df.to_excel('photo_list.xlsx')